In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import respy as rp
import numpy as np
from estimagic.optimization.optimize import maximize

<IPython.core.display.Javascript object>

In [3]:
# Parameters for the simulation --> will go to config.py
NUM_AGENTS = 500
NUM_PERIODS = 5

<IPython.core.display.Javascript object>

### Working Step 1: Model estimation via Maximum Likelihood

- We estimate a model by Maximum Likelihood estimation using all true values as the starting point. We start with a model that does not include ambiguity, $\eta = 0$. 
- We do not fix any of the values and use a gradient-free optimizer.

In [4]:
params_base, options = rp.get_example_model("robinson_crusoe_basic", with_data=False)

<IPython.core.display.Javascript object>

We will simulate NUM_AGENTS representative Robinson Crusoe agents over a time of T = NUM_PERIODS periods.

In [5]:
options["n_periods"] = NUM_PERIODS
options["simulation_agents"] = NUM_AGENTS

<IPython.core.display.Javascript object>

In [6]:
simulate = rp.get_simulate_func(params_base, options)
df = simulate(params_base)
# df

<IPython.core.display.Javascript object>

**Get the criterion function** and output the value of the MLE at given parameter values.

**Note**  
We obtain a version of the likelihood function where all arguments - except for the parameter vector - are fixed. 
Consequently, the function can be directly passed into an optimizer.

$\eta$ enters through the `model_processing._parse_parameters` method, which was added through [PR #349](https://github.com/OpenSourceEconomics/respy/pull/349) into `respy`.

In [7]:
crit_func = rp.get_crit_func(params_base, options, df)

<IPython.core.display.Javascript object>

In [8]:
rslt, params_rslt = maximize(
    criterion=crit_func,
    params=params_base,
    algorithm="nlopt_bobyqa",
    algo_options={"maxeval": 20},
    # for the first tries we set the evaluations to a low number
    dashboard=False,
)

<IPython.core.display.Javascript object>

In [9]:
# params_rslt

<IPython.core.display.Javascript object>

### Working step 2: Ambiguity perturbation

We will disturb $\eta$ and subsequently estimate it to check whether the same values come out again.

More specifically:
* We will first set $\eta$ so the values presented at p.17 in Eisenhauer and Suchy (2020), i.e. baseline (0.01) and high (0.02)
* We will simulate the model with the set values
* We will perturb our starting values in the estimation and analyze whether they return to their initial values

In [10]:
eta_values = {
    "risk_only": 0.00,
    "baseline": 0.01,
    "high": 0.02,
}

<IPython.core.display.Javascript object>

In [11]:
params_eta = params_base.copy()
params_eta.loc[("eta", "eta"), :] = eta_values["baseline"]
params_eta

value
category                name                          
delta                   delta                     0.95
wage_fishing            exp_fishing               0.10
nonpec_fishing          constant                 -1.00
nonpec_hammock          constant                  2.50
                        not_fishing_last_period  -1.00
shocks_sdcorr           sd_fishing                1.00
                        sd_hammock                1.00
                        corr_hammock_fishing     -0.20
lagged_choice_1_hammock constant                  1.00
inadmissibility_penalty inadmissibility_penalty -20.00
eta                     eta                       0.01

<IPython.core.display.Javascript object>

Initialize the base parameter constraints. Base refers here to the case where all parameters are fixed.

In [12]:
# constr = [{"query": "category != 'eta'", "type": "fixed"}]
constr_base = [
    {"loc": "shocks_sdcorr", "type": "sdcorr"},
    {"loc": "eta", "type": "fixed"},
    {"loc": "delta", "type": "fixed"},
    {"loc": "wage_fishing", "type": "fixed"},
    {"loc": "nonpec_fishing", "type": "fixed"},
    {"loc": "nonpec_hammock", "type": "fixed"},
    {"loc": "shocks_sdcorr", "type": "fixed"},
]

<IPython.core.display.Javascript object>

We will keep all parameters fixed, but let $\eta$ vary during the estimation.

In [13]:
constr_eta = constr_base.copy()
constr_eta.remove({"loc": "eta", "type": "fixed"})

<IPython.core.display.Javascript object>

In [14]:
# Check the parameter combinations
eta_perturbed = {"true": [], "estimated": [], "yos": []}

<IPython.core.display.Javascript object>

**Get a simulated data set** where agents act accordingly to $\eta = 0.01$.

In [15]:
simulate_eta = rp.get_simulate_func(params_eta, options)
df_eta = simulate_eta(params_eta)
# df_eta

<IPython.core.display.Javascript object>

In [16]:
for eta in np.linspace(0, 0.25, 11):

    # We want a change in the starting values (not in the simulated data)
    params_eta.loc[("eta", "eta"), :] = eta

    # params["group"] = params.index.get_level_values('category')
    # need above only if we want dashboard output

    # Get the criterion function with simulated data, but different param values
    crit_func_eta = rp.get_crit_func(params_eta, options, df_eta)

    results, params_rslt = maximize(
        crit_func_eta,
        params_eta,
        "nlopt_bobyqa",
        algo_options={"maxeval": 20},
        # In the "real world simulation" we will set this to a much higher number
        constraints=constr_eta,
    )

    eta_perturbed["true"].append(eta)
    eta_perturbed["estimated"].append(params_rslt.loc["eta", "value"][0])
    # eta_perturbed["yos"].append(stat)

    # To do: write a wrapper for this thing

<IPython.core.display.Javascript object>

In [17]:
eta_perturbed

{'true': [0.0,
  0.025,
  0.05,
  0.07500000000000001,
  0.1,
  0.125,
  0.15000000000000002,
  0.17500000000000002,
  0.2,
  0.225,
  0.25],
 'estimated': [0.006404720942368252,
  0.0,
  0.004545104350805318,
  0.00712175520172691,
  0.007071067811865473,
  0.004226356036088265,
  0.007377843102020493,
  0.007981338325262889,
  0.00504741850065829,
  0.008653465870489294,
  0.00840140733006331],
 'yos': []}

<IPython.core.display.Javascript object>

### Working step 3: Risk-only model

Write explicitly what is fixed.


We estimate a model with *risk-only*. This means we have $\eta = 0$  fixed for the simulated (estimation).
We want to see how different the likelihood is at the estimated parameter values.
We treat the risk-parameter as only unknown parameter.

In [18]:
params_risk_only = params_base.copy()
params_risk_only.loc[("eta", "eta"), :] = eta_values["risk_only"]
params_risk_only

value
category                name                          
delta                   delta                     0.95
wage_fishing            exp_fishing               0.10
nonpec_fishing          constant                 -1.00
nonpec_hammock          constant                  2.50
                        not_fishing_last_period  -1.00
shocks_sdcorr           sd_fishing                1.00
                        sd_hammock                1.00
                        corr_hammock_fishing     -0.20
lagged_choice_1_hammock constant                  1.00
inadmissibility_penalty inadmissibility_penalty -20.00
eta                     eta                       0.00

<IPython.core.display.Javascript object>

In [19]:
constr_risk_only = constr_base.copy()
constr_risk_only.remove({"loc": "delta", "type": "fixed"})

<IPython.core.display.Javascript object>

In [20]:
simulate_risk_only = rp.get_simulate_func(params_risk_only, options)
df_risk_only = simulate_risk_only(params_risk_only)

<IPython.core.display.Javascript object>

In [21]:
crit_func_risk_only = rp.get_crit_func(params_risk_only, options, df_risk_only)
crit_func(params_risk_only)

-6.803133136863534

<IPython.core.display.Javascript object>

In [22]:
results, params_rslt = maximize(
    crit_func_risk_only,
    params_risk_only,
    "nlopt_bobyqa",
    algo_options={"maxeval": 40},
    constraints=constr_risk_only,
    dashboard=False,
)

<IPython.core.display.Javascript object>

In [23]:
# params_rslt

<IPython.core.display.Javascript object>

### Working Step 4: 

In the following we will simulate data with $\eta = 0$ and take a try whether we will estiamte $\eta = 0$ again.

In [24]:
params_no_ambiguity = params_base.copy()
params_no_ambiguity.loc[("eta", "eta"), :] = eta_values["risk_only"]

<IPython.core.display.Javascript object>

In [25]:
constr_no_ambiguity = constr_base.copy()
constr_no_ambiguity.remove({"loc": "eta", "type": "fixed"})

<IPython.core.display.Javascript object>

In [26]:
simulate_no_ambiguity = rp.get_simulate_func(params_no_ambiguity, options)
df_no_ambiguity = simulate_no_ambiguity(params_no_ambiguity)

<IPython.core.display.Javascript object>

In [27]:
eta_estimated_no_ambiguity = []

<IPython.core.display.Javascript object>

In [28]:
for eta in np.linspace(0, 0.25, 11):

    params_no_ambiguity.loc[("eta", "eta"), :] = eta

    crit_func_no_ambiguity = rp.get_crit_func(
        params_no_ambiguity, options, df_no_ambiguity
    )

    results, params_rslt = maximize(
        crit_func_no_ambiguity,
        params_no_ambiguity,
        "nlopt_bobyqa",
        algo_options={"maxeval": 140},
        constraints=constr_no_ambiguity,
        dashboard=False,
    )

    eta_estimated_no_ambiguity.append(params_rslt.loc["eta", "value"][0])

<IPython.core.display.Javascript object>

In [29]:
# params_rslt

<IPython.core.display.Javascript object>

### Summary of working steps

* We estimated a model by maximum likelihood using all true values as the starting point.

* We estimated a model with only $\eta$ perturbed.

* We estimated a model with risk-only, $\eta = 0$ fixed, to see how different the likelihood is at the estimated parameter values.